En este cuardenos vamos a crear unos métodos para crear la base de datos o actualizarla si ha habido cambios.

In [106]:
def texto_correcto(texto):
    num = len([x for x in texto if not x.isalnum() and x not in ['.', ',', ';', ':', ' ', '?', '¿', '¡', '!', '(', ')']])
    return num < 10, num

def get_chunks(texto, tam_chunks = 1000):
    ultimo_punto = 0
    inicio_chunk = 0
    posicion_texto = 0
    contador_chunk = 0
    aux = []
    resultado = []
    while posicion_texto < len(texto):
        if contador_chunk == 0:
            chunk = ''
        chunk += texto[posicion_texto]
        if texto[posicion_texto] == '.':
            ultimo_punto = posicion_texto
        posicion_texto += 1
        contador_chunk += 1
        if contador_chunk == tam_chunks:
            aux.append(chunk)
            contador_chunk = 0
            if posicion_texto != ultimo_punto:
                if inicio_chunk < ultimo_punto:
                    posicion_texto = ultimo_punto+2                    
            inicio_chunk = posicion_texto
    if contador_chunk != 0: 
        aux.append(chunk)
    
    for chunk in aux:
        alnum = sum(c.isalnum() for c in chunk)
        letra = sum(c.isalpha() for c in chunk)
        todos = len(chunk)
        if  alnum/todos > 0.78 and len(chunk) > tam_chunks*0.25 and texto_correcto(chunk)[0] and letra/alnum > 0.9:
            resultado.append(chunk)
    return resultado

In [6]:
import re
import io
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt

def eliminarDivisionesSilabicas(s):
    palabrasObejtivo = re.findall(r"\w+-\n\w+",s)
    sOut = s
    for m in palabrasObejtivo:
        new = m.replace("-\n","")
        sOut = sOut.replace(m,new)
    return sOut

def corregir_indice(s):
    return re.sub(r"\.\.\.\.+", '.', s)

def convertirPDF(nombreFichero):
    numPag = set()
    output = io.StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(nombreFichero, 'rb')
    for page in PDFPage.get_pages(infile, numPag):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    text = eliminarDivisionesSilabicas(corregir_indice(text)).replace("\n", " ").replace("\x0c", " ")
    return " ".join(text.split())

In [108]:
import hashlib
import pandas as pd
import glob

def crearBaseDatos(ruta_documentos = "Documentos"):
    contador = 1
    chunks = []
    checksums = []
    documentos = glob.glob("Documentos/*")
    m = len(documentos)
    for file in documentos:
        fichero = file.split("/")[1]
        checksums += [(hashlib.md5(open(file,'rb').read()).hexdigest(), fichero)]
        # Extraemos el texto del PDF
        texto = convertirPDF(file)
        chunks += [(elem, fichero) for elem in get_chunks(texto)]
        
        print(f'Documento {contador} de {m}', end="\r")
        contador += 1

    pd.DataFrame(chunks, columns=['Contexto', 'Documento']).to_csv("Contextos.csv", sep=',', index=False)
    pd.DataFrame(checksums, columns=['Checksum', 'Documento']).to_csv("Checksums.csv", sep=',', index=False)

In [1]:
import pandas as pd

def actualizarBaseDatos(ruta_documentos = "Documentos"):
    ficheros = glob.glob("Documentos/*")
    documentos = [elem.split("/")[1] for elem in ficheros]
    df_contextos = pd.read_csv('Contextos.csv')
    df_checksums = pd.read_csv('Checksums.csv')
    
    documentos_persistidos = df_checksums['Documento'].tolist()
    documentos_checksums = df_checksums['Checksum'].tolist()
    # Primero comprobamos si hay ficheros nuevos
    documentos_nuevos = list(set(documentos) - set(documentos_persistidos))
    checksums, chunks = [], []
    for nuevo in documentos_nuevos:
        checksum = hashlib.md5(open(f'Documentos/{nuevo}','rb').read()).hexdigest()
        if checksum in documentos_checksums:
            nombre_viejo = df_checksums[df_checksums["Checksum"] == checksum].Documento.values[0]
            df_checksums.loc[df_checksums.Checksum == checksum, 'Documento'] = nuevo
            df_contextos.loc[df_contextos.Documento == nombre_viejo, 'Documento'] = nuevo
        else:
            checksums += [(hashlib.md5(open(f'Documentos/{nuevo}','rb').read()).hexdigest(), nuevo)]
            texto = convertirPDF(f'Documentos/{nuevo}')
            chunks += [(elem, nuevo) for elem in get_chunks(texto)]
    df_contextos = pd.concat([df_contextos, pd.DataFrame(chunks, columns=['Contexto', 'Documento'])])
    df_checksums = pd.concat([df_checksums, pd.DataFrame(checksums, columns=['Checksum', 'Documento'])])
    
    # Segundo eliminamos los que ya no estén entre los documentos
    documentos_eliminar = list(set(documentos_persistidos) - set(documentos))
    for eliminar in documentos_eliminar:
        df_contextos.drop(df_contextos.loc[df_contextos['Documento']==eliminar].index, inplace=True)
        df_checksums.drop(df_checksums.loc[df_checksums['Documento']==eliminar].index, inplace=True)
    
    # Persistimos los cambios
    df_contextos.sort_values('Documento').to_csv("Contextos.csv", sep=',', index=False)
    df_checksums.sort_values('Documento').to_csv("Checksums.csv", sep=',', index=False)
    print("Operación completada")